In [ ]:
from langchain.chat_models import ChatOpenAI

# from langchain.prompts import example_selector
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


# ramdom example selector
class RandomExampleSelector(BaseExampleSelector):
    # examples를 받아서 초기화
    def __init__(self, examples: list[dict[str, any]]):
        self.examples = examples

    # exmaples에 example을 추가하는 method
    def add_example(self, example):
        self.examples.append(example)

    # examples 중 무작위로 하나의 example을 선택해 반환함
    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


# PromptTemplate을 이용하여 형식을 지정
example_prompt = PromptTemplate.from_template("Human: {country}\nAI:{answer}")

# examples를 조건에 맞추어 선택하도록 할 수 있음(예제의 양을 줄여 비용을 절감할 수 있음)
example_selector = RandomExampleSelector(examples=examples)

# 랭체인이 알아서 example_selector에서 선정된 예제를 사용하여 prompt를 형식화함
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}",
    input_variables=["country"],
)

# 위에까지가 언어 모델(LM)이 완성된 것임

prompt.format(country="Brazil")

TypeError: string indices must be integers, not 'str'